##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFX Python 関数コンポーネントチュートリアル


注：この例は、Jupyter スタイルのノートブックで今すぐ実行できます。セットアップは必要ありません。「Google Colab で実行」をクリックするだけです

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/python_function_component"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で実行</a></td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tfx/tutorials/tfx/python_function_component.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colabで実行</a>
</td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tfx/tutorials/tfx/python_function_component.ipynb">     <img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     GitHubでソースを表示</a></td>
<td><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tfx/tutorials/tfx/python_function_component.ipynb"><img width="32px" src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table></div>

このノートブックには、TFX Interactive Context 内およびローカルで調整された TFX パイプラインで Python 関数コンポーネントを作成および実行する方法の例が含まれています。

詳細なコンテキストと情報については、TFX ドキュメントサイトの[カスタム Python 関数コンポーネント](https://www.tensorflow.org/tfx/guide/custom_function_component)ページを参照してください。

## MNIST モデルをビルドする

最初に TFX をインストールし、必要なモジュールをインポートします。TFX には Python 3 が必要です。

### システムの Python バージョンを確認する


In [ ]:
import sys
sys.version

### Pip のアップグレード

ローカルで実行する場合にシステム Pip をアップグレードしないようにするには、Colab で実行していることを確認してください。もちろん、ローカルシステムは個別にアップグレードできます。

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### TFX をインストールする

**注意：Google Colab では、パッケージが更新されるため、このセルを初めて実行するときに、ランタイムを再起動（[ランタイム] &gt; [ランタイムの再起動...]）する必要があります。**

In [ ]:
!pip install tfx

## ランタイムを再起動しましたか？

Google Colab を使用している場合は、上記のセルを初めて実行するときにランタイムを再起動（[ランタイム]　&gt; [ランタイムの再起動...]）する必要があります。これは、Colab がパッケージを読み込むために必要です。

### パッケージをインポートする

TFX をインポートしてバージョンを確認します。

In [ ]:
# Check version
from tfx import v1 as tfx
tfx.__version__

## カスタム Python 関数コンポーネント

このセクションでは、Python 関数からコンポーネントを作成します。実際の ML の問題は発生しません。これらの単純な関数は、Python 関数コンポーネントの開発プロセスを説明するためにのみ使用されます。

詳細なドキュメントについては、[Python 関数ベースのコンポーネントガイド](https://www.tensorflow.org/tfx/guide/custom_function_component)を参照してください。

### Python カスタムコンポーネントを作成する

最初に、ダミーデータを生成する関数を作成します。これは、独自の Python モジュールファイルに書き込まれます。

In [ ]:
%%writefile my_generator.py

import os
import tensorflow as tf  # Used for writing files.

from tfx import v1 as tfx

# Non-public APIs, just for showcase.
from tfx.types.experimental.simple_artifacts import Dataset

@tfx.dsl.components.component
def MyGenerator(data: tfx.dsl.components.OutputArtifact[Dataset]):
  """Create a file with dummy data in the output artifact."""
  with tf.io.gfile.GFile(os.path.join(data.uri, 'data_file.txt'), 'w') as f:
    f.write('Dummy data')

  # Set metadata and ensure that it gets passed to downstream components.
  data.set_string_custom_property('my_custom_field', 'my_custom_value')

次に、生成されたダミーデータを使用する 2 番目のコンポーネントを記述します。データのハッシュを計算して返します。

In [ ]:
%%writefile my_consumer.py

import hashlib
import os
import tensorflow as tf

from tfx import v1 as tfx

# Non-public APIs, just for showcase.
from tfx.types.experimental.simple_artifacts import Dataset
from tfx.types.standard_artifacts import String

@tfx.dsl.components.component
def MyConsumer(data: tfx.dsl.components.InputArtifact[Dataset],
               hash: tfx.dsl.components.OutputArtifact[String],
               algorithm: tfx.dsl.components.Parameter[str] = 'sha256'):
  """Reads the contents of data and calculate."""
  with tf.io.gfile.GFile(
      os.path.join(data.uri, 'data_file.txt'), 'r') as f:
    contents = f.read()
  h = hashlib.new(algorithm)
  h.update(tf.compat.as_bytes(contents))
  hash.value = h.hexdigest()

  # Read a custom property from the input artifact and set to the output.
  custom_value = data.get_string_custom_property('my_custom_field')
  hash.set_string_custom_property('input_custom_field', custom_value)

### InteractiveContext を使用してノートブックで実行する

次に、TFX InteractiveContext での新しいコンポーネントの使用法を示します。

TFX ノートブック InteractiveContext でできることの詳細については、ノートブック内の [TFS Keras コンポーネントチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras)を参照してください。

In [ ]:
from my_generator import MyGenerator
from my_consumer import MyConsumer

#### InteractiveContext を作成する

In [ ]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
context = InteractiveContext()

#### `context.run()` を使用してコンポーネントをインタラクティブに実行する

次に、`context.run()` を使用して、ノートブック内でコンポーネントをインタラクティブに実行します。私たちのコンシューマーコンポーネントは、ジェネレーターコンポーネントの出力を使用します。

In [ ]:
generator = MyGenerator()
context.run(generator)

In [ ]:
consumer = MyConsumer(
    data=generator.outputs['data'],
    algorithm='md5')
context.run(consumer)

実行後、ディスク上のコンシューマーコンポーネントの「ハッシュ」出力アーティファクトの内容を検査できます。

In [ ]:
!tail -v {consumer.outputs['hash'].get()[0].uri}

これで、独自のカスタムコンポーネントを作成して実行できました。

### パイプライン定義を記述する

次に、これらの同じコンポーネントを使用してパイプラインを作成します。ノートブック内での `InteractiveContext` の使用は実験には適していますが、パイプラインを定義することでパイプラインをローカルまたはリモートのランナーにデプロイして本番環境で使用できます。

ここでは、マシン上でローカルに実行されている LocalDagRunner の使用法を示します。本番環境での実行には、Airflow または Kubeflow ランナーの方が適している場合があります。

#### パイプラインを作成する

In [ ]:
import os
import tempfile
from tfx import v1 as tfx

# Select a persistent TFX root directory to store your output artifacts.
# For demonstration purposes only, we use a temporary directory.
PIPELINE_ROOT = tempfile.mkdtemp()
# Select a pipeline name so that multiple runs of the same logical pipeline
# can be grouped.
PIPELINE_NAME = "function-based-pipeline"
# We use a ML Metadata configuration that uses a local SQLite database in
# the pipeline root directory. Other backends for ML Metadata are available
# for production usage.
METADATA_CONNECTION_CONFIG = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    os.path.join(PIPELINE_ROOT, 'metadata.sqlite'))

def function_based_pipeline():
  # Here, we construct our generator and consumer components in the same way.
  generator = MyGenerator()
  consumer = MyConsumer(
      data=generator.outputs['data'],
      algorithm='md5')

  return tfx.dsl.Pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      components=[generator, consumer],
      metadata_connection_config=METADATA_CONNECTION_CONFIG)

my_pipeline = function_based_pipeline()

#### `LocalDagRunner` でパイプラインを実行する

In [ ]:
tfx.orchestration.LocalDagRunner().run(my_pipeline)

このパイプラインの実行によって生成された出力アーティファクトを検査できます。

In [ ]:
!find {PIPELINE_ROOT}

これで、独自のカスタムコンポーネントを作成し、LocalDagRunner での実行を調整しました。次のステップについては、[TFX Webサイト](https://www.tensorflow.org/tfx)で追加のチュートリアルとガイドを確認してください。